In [ ]:
import sys


sys.path.append("../src/")

In [ ]:
from sennet.core.mmap_arrays import read_mmap_array, create_mmap_array
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import cc3d

In [ ]:
# labels_out = cc3d.connected_components(labels_in) # 26-connected
# labels_out = cc3d.dust(
#     labels_in, 
#     threshold=100, 
#     connectivity=26, 
#     in_place=False,
# )

In [ ]:
from pathlib import Path


# folder = "kidney_3_dense"
folder = "kidney_2"
root_path = Path(f"/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/{folder}/")
mmap_paths = sorted([p for p in root_path.glob("chunk_*")])
for p in mmap_paths:
    print(p)
mmaps = [read_mmap_array(p / "thresholded_prob")for p in mmap_paths]
print([m.shape for m in mmaps])
mmap = np.concatenate([m.data for m in mmaps], axis=0)
# mmap = read_mmap_array("/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_02/thresholded_prob")

In [ ]:
connectivity = 26 # only 4,8 (2D) and 26, 18, and 6 (3D) are allowed
labels_out, num_labels = cc3d.connected_components(
    mmap, 
    connectivity=connectivity, 
    # out_file="out.bin",
    return_N=True,
)
labels_out = cc3d.dust(
    labels_out, 
    threshold=1000,
    connectivity=26, 
    in_place=False,
)

In [ ]:
!ls -lah

In [ ]:
# unique_labels = np.unique(labels_out)

In [ ]:
from tqdm import tqdm


out_mmap = create_mmap_array(
    Path(f"/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled_cc3d/{folder}/chunk_00/thresholded_prob/"),
    mmap.shape,
    bool,
)
# out_mmap.data[:] = out_image

# for label, image in tqdm(cc3d.each(labels_out, binary=True, in_place=True), total=len(unique_labels)):
for label, image in tqdm(cc3d.each(labels_out, binary=True, in_place=True)):
    out_mmap.data[image] = True
    # out_image = out_image | image
    # out_image = out_image | (image > 0)
out_mmap.data.flush()


In [ ]:
from sennet.core.submission_utils import generate_submission_df_from_one_chunked_inference

In [ ]:
pred_dir = Path(f"/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled_cc3d/{folder}")
sub_df = generate_submission_df_from_one_chunked_inference(pred_dir)
sub_df.to_csv(pred_dir / "submission.csv")

In [ ]:
# image

In [ ]:
out_mmap.data.sum() / mmap.sum()